#### Libraries

In [13]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
import gensim

import sklearn.preprocessing as preproc
from sklearn.feature_extraction import text
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, plot_confusion_matrix, ConfusionMatrixDisplay, confusion_matrix

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, Dense, Dropout, MaxPooling1D
from tensorflow import keras
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import layers
from scikeras.wrappers import KerasClassifier
from gc import callbacks

import pickle
import warnings
import numpy as np

#### Data preprocessing

In [14]:
data_df = pd.read_csv('./data.csv')

In [15]:
data_df

,font_size,fontname,text,has_fullstop,text_length,x0,x1,y0,y1,height,size_diff_prev,xdiff,ydiff,isHeader,headerLevel
0,17,Symmetries in quantum field theory\n,Symmetries in quantum field theory,False,35,181.252,429.988707,612.034997,629.250397,17.215400,NaN,NaN,NaN,True,1.0
1,12,Jiwoo Park\n,Jiwoo Park,False,11,276.849,334.398942,584.181691,596.136891,11.955200,-5.0,9.559700e+01,-5.260200,NaN,NaN
2,12,November 2021\n,November 2021,False,14,266.281,344.970126,560.735691,572.690891,11.955200,0.0,-1.056800e+01,0.000000,NaN,NaN
3,14,1 Noether’s theorem\n,1 Noether’s theorem,False,20,133.768,289.985206,517.363837,531.710037,14.346200,2.0,-1.325130e+02,2.391000,True,2.0
4,10,→\n,Classical Noether’s theorem. Say that we have ...,True,140,133.768,477.479600,484.438256,506.355856,21.917600,-4.0,-2.842171e-14,7.571400,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3038,10,ASYTPY+CMBX10,"[48] J. Polchinski, Eﬀective ﬁeld theory and t...",True,230,111.969,483.306504,253.996350,288.427856,34.431506,0.0,-2.047900e+01,24.468906,NaN,NaN
3039,10,KFNGAX+CMR10,(1996) [hep-ph/9512380].,True,25,132.448,241.481683,242.041350,252.003950,9.962600,0.0,2.047900e+01,-24.468906,NaN,NaN
3040,10,EYSKRI+CMTI10,"[51] J. Wess and J. Bagger, Supersymmetry and ...",True,81,111.969,483.310499,230.086350,240.606856,10.520506,0.0,-2.047900e+01,0.557906,NaN,NaN
3041,10,KFNGAX+CMR10,"Press, USA, 1992).",True,19,132.448,214.643435,218.131350,228.093950,9.962600,0.0,2.047900e+01,-0.557906,NaN,NaN


In [16]:
def clean_text(text):
    '''Text preprocessing/cleaning function'''
    
    # Convert to lower case
    text = text.lower()
    
    # Remove symbols, special characters
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    
    # Remove stop words
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)

    # Tokenize 
    text =  nltk.WordPunctTokenizer().tokenize(text)
        
    return text
data_df['Cleaned'] = list(map(clean_text, data_df.text))

In [17]:
X_train = data_df['Cleaned']
y_train = data_df[['isHeader', 'headerLevel']].replace(np.nan,-1).replace(True,1)
y_train['isHeader'] = y_train['isHeader'].replace(-1,0)

In [18]:
data_df.text

0                     Symmetries in quantum field theory 
1                                             Jiwoo Park 
2                                          November 2021 
3                                    1 Noether’s theorem 
4       Classical Noether’s theorem. Say that we have ...
                              ...                        
3038    [48] J. Polchinski, Eﬀective ﬁeld theory and t...
3039                            (1996) [hep-ph/9512380]. 
3040    [51] J. Wess and J. Bagger, Supersymmetry and ...
3041                                  Press, USA, 1992). 
3042    [52] L. F. Abbott, Nucl. Phys. B 185, 189 (198...
Name: text, Length: 3043, dtype: object

In [19]:
y_train

,isHeader,headerLevel
0,1,1.0
1,0,-1.0
2,0,-1.0
3,1,2.0
4,0,-1.0
...,...,...
3038,0,-1.0
3039,0,-1.0
3040,0,-1.0
3041,0,-1.0


In [24]:

w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=256,
                                   window=4,
                                   min_count=1)
w2v_model.wv['lstm'].shape

(256,)

In [21]:
words = set(w2v_model.wv.index_to_key)

X_train_vec = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train])

/tmp/ipykernel_79104/1965191243.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vec = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])


In [10]:
X_train.shape

(3043,)

In [11]:
X_train_vec[4].shape

(17, 256)

In [12]:
# TODO: need to pad the word vec's so they have equal length

#### Feaure engineering

#### Dummy data to verify


In [ ]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

def clean_text(text):
    '''Text preprocessing/cleaning function'''
    
    # Convert to lower case
    text = text.lower()
    
    # Remove symbols, special characters
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    
    # Remove stop words
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)

    # Tokenize 
    text =  nltk.WordPunctTokenizer().tokenize(text)
        
    return text

train_df['Cleaned'] = list(map(clean_text, train_df.text))
test_df['Cleaned'] = list(map(clean_text, test_df.text))

X_train, y_train = train_df['Cleaned'], train_df['label']
X_test, y_test = test_df['Cleaned'], test_df['label']

w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=256,
                                   window=4,
                                   min_count=1)

words = set(w2v_model.wv.index_to_key)

X_train_vec = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train])

X_test_vec = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test])

X_train = []
for v in X_train_vec:
    if v.size:
        X_train.append(v.mean(axis=0))
    else:
        X_train.append(np.zeros(256, dtype=float))
  
X_test = []
for v in X_test_vec:
    if v.size:
        X_test.append(v.mean(axis=0))
    else:
        X_test.append(np.zeros(256, dtype=float))

#### Machine learning 

In [ ]:
# Logger

import wandb
from wandb.keras import WandbCallback

wandb.login()

In [ ]:
labels=['0', '1']

# Logistic regression 

param_log = {'solver':['lbfgs'],
              'C':[1, 5, 10, 50, 100]}

# Create model
logr_clf = LogisticRegression(max_iter=10000)

# 10 fold cv hyper-parameters tuning
clf_log = GridSearchCV(logr_clf,
                       param_grid=param_log, 
                       cv=10, 
                       scoring='accuracy',
                       refit=True) 

# Fit
clf_log.fit(X_train, y_train)

print("Best parameters", clf_log.best_params_)
print("Best score", clf_log.best_score_)

# Predict on test
logr_y_pred = clf_log.predict(X_test)
logr_y_proba = clf_log.predict_proba(X_test)

print('Accuracy Score: ' + str(accuracy_score(y_test,logr_y_pred)))
print('Classification report\n', classification_report(y_test, logr_y_pred)) 

wandb.init(project="Title classifier", reinit=True)
wandb.run.name = 'Logistic regression'
wandb.sklearn.plot_classifier(clf_log, X_train, X_test, y_train, y_test, logr_y_pred, logr_y_proba, labels,
                                                         model_name='Logistic regression', feature_names=None)
wandb.run.save
wandb.run.finish()

In [ ]:

# Svc

param_svc = {'kernel':['rbf'],
              'C':[0.1, 1, 5, 10],
              'gamma': [0.01, 0.1, 1, 5]}

# Create model
svc_clf = SVC(probability=True)

# 10 fold cv hyper-parameters tuning
clf_svc = GridSearchCV(svc_clf,
                       param_grid=param_svc, 
                       cv=10, 
                       scoring='accuracy',
                       refit=True) 
# Fit
clf_svc.fit(X_train, y_train)

print("Best parameters", clf_svc.best_params_)
print("Best score", clf_svc.best_score_)

# Predict on test
svc_y_pred = clf_svc.predict(X_test)
svc_y_proba = clf_svc.predict_proba(X_test)

print('Accuracy Score: ' + str(accuracy_score(y_test,svc_y_pred)))
print('Classification report\n', classification_report(y_test, svc_y_pred)) 

wandb.init(project="Title classifier", reinit=True)
wandb.run.name = 'Support vector'
wandb.sklearn.plot_classifier(clf_svc, X_train, X_test, y_train, y_test, svc_y_pred, svc_y_proba, labels,
                                                         model_name='Support vector', feature_names=None)
wandb.run.save
wandb.run.finish()

In [ ]:

# Random Forest

param_rf = { 
    'n_estimators': [5, 10, 20, 50, 100, 150],
    'max_features': ['sqrt', 'log2'],
    'max_depth' : [1,2,3,4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

rf_clf = RandomForestClassifier(random_state=41)

clf_rf = GridSearchCV(estimator=rf_clf, 
                      param_grid=param_rf, 
                      scoring='accuracy',
                      cv=10,
                      refit=True)

clf_rf.fit(X_train, y_train)

print("Best parameters", clf_rf.best_params_)
print("Best score", clf_rf.best_score_)

rfc_y_pred = clf_rf.predict(X_test)
rfc_y_proba = clf_rf.predict_proba(X_test)

print('Accuracy Score: ' + str(accuracy_score(y_test, rfc_y_pred)))
print("Classification report: \n", classification_report(y_test, rfc_y_pred))

wandb.init(project="Title classifier", reinit=True)
wandb.run.name = 'Random forest'
wandb.sklearn.plot_classifier(clf_rf, X_train, X_test, y_train, y_test, rfc_y_pred, rfc_y_proba, labels,
                                                         model_name='Random forest', feature_names=None)
wandb.run.save
wandb.run.finish()

In [ ]:

# Decision tree

param_dt = {'max_features': ['sqrt', 'log2'],
              'ccp_alpha': [0.5, 0.1, .01, .001],
              'max_depth' : [1,2,3,4,5,6,7,8,9],
              'criterion' :['gini', 'entropy']
             }

dt_clf = DecisionTreeClassifier()

clf_dt = GridSearchCV(dt_clf, 
                      param_grid=param_dt, 
                      scoring='accuracy',
                      cv=10,
                      refit=True)

clf_dt.fit(X_train, y_train)

print("Best parameters", clf_dt.best_params_)
print("Best score", clf_dt.best_score_)

dt_y_pred = clf_dt.predict(X_test)
dt_y_proba = clf_dt.predict_proba(X_test)

print('Accuracy Score: ' + str(accuracy_score(y_test, dt_y_pred)))
print("Classification report: \n", classification_report(y_test, dt_y_pred))

wandb.init(project="Title classifier", reinit=True)
wandb.run.name = 'Decision tree'
wandb.sklearn.plot_classifier(clf_dt, X_train, X_test, y_train, y_test, dt_y_pred, dt_y_proba, labels,
                                                         model_name='Decision tree', feature_names=None)
wandb.run.save
wandb.run.finish()

In [ ]:
# KNN

param_knn = { 
    'n_neighbors': list(range(1,11)),
}

knn_clf = KNeighborsClassifier()

clf_knn = GridSearchCV(estimator=rf_clf, 
                      param_grid=param_rf, 
                      scoring='accuracy',
                      cv=10,
                      refit=True)

clf_knn.fit(X_train, y_train)

print("Best parameters", clf_knn.best_params_)
print("Best score", clf_knn.best_score_)

knn_y_pred = clf_knn.predict(X_test)
knn_y_proba = clf_knn.predict_proba(X_test)

print('Accuracy Score: ' + str(accuracy_score(y_test, knn_y_pred)))
print("Classification report: \n", classification_report(y_test, knn_y_pred))

wandb.init(project="Title classifier", reinit=True)
wandb.run.name = 'KNN'
wandb.sklearn.plot_classifier(clf_knn, X_train, X_test, y_train, y_test, knn_y_pred, knn_y_proba, labels,
                                                         model_name='KNN', feature_names=None)
wandb.run.save
wandb.run.finish()

In [ ]:
# XGB

param_xgb = {
        'n_estimators': range(20, 400, 20),
        'learning_rate': [1e-1, 1e-2, 5e-3, 5e-4],
        'min_child_weight': [1, 3, 5, 7, 9],
        'gamma': [0.5, 1, 1.5, 2],
        'subsample': [0.5, 0.75, 1.0],
        'colsample_bytree': [0.5, 0.75, 1.0],
        'max_depth': [2, 3, 4, 5, 6, 7, 8]
        }

xgb_clf = XGBClassifier(objective='binary:logistic', seed=41)

clf_xgb = GridSearchCV(estimator=xgb_clf, 
                      param_grid=param_xgb, 
                      scoring='accuracy',
                      cv=10,
                      refit=True)

clf_xgb.fit(X_train, y_train)

print("Best parameters", clf_xgb.best_params_)
print("Best score", clf_xgb.best_score_)

xgb_y_pred = clf_xgb.predict(X_test)
xgb_y_proba = clf_xgb.predict_proba(X_test)

print('Accuracy Score: ' + str(accuracy_score(y_test, xgb_y_pred)))
print("Classification report: \n", classification_report(y_test, xgb_y_pred))

wandb.init(project="Title classifier", reinit=True)
wandb.run.name = 'XGB'
wandb.sklearn.plot_classifier(clf_xgb, X_train, X_test, y_train, y_test, xgb_y_pred, xgb_y_proba, labels,
                                                         model_name='XGB', feature_names=None)
wandb.run.save
wandb.run.finish()

#### Deep learning

##### Tuning

In [ ]:
embed_len = 512
max_tokens = 300
max_words = 100
n_classes = 2

sweep_config = {
   'method': 'grid',
   
   'parameters': {
       
       'neurons': {
           'values': [32, 64, 128]
       },
       
       'f': {
           'values': [64, 128, 256]
       },
       
       'bs': {
           'values': [8, 16, 32, 64]
       }
   }
}

wandb.init(project="Title classifier", reinit=True)
wandb.run.name = 'CNN model tuning'

sweep_id = wandb.sweep(sweep_config)

In [ ]:
def train():
       
   configs = {
       'neurons': 32,
       'f': 64,
       'bs': 8
   }
   
   # Specify the other hyperparameters to the configuration
   config = wandb.config
   config.epochs = 50
   
   model = keras.Sequential(
        [
            keras.Input(shape=(max_tokens, )),
            
            layers.Embedding(input_dim=max_words, 
                             output_dim=embed_len,  
                             input_length=max_tokens),
            
            # Block 1
            layers.Conv1D(wandb.config.f, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling1D(pool_size=(3, 3)),
            layers.Dropout(0.3),                  
            
            # Block 2
            layers.Conv1D(wandb.config.f, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling1D(pool_size=(3, 3)),
            layers.Dropout(0.3),                
            
            # FC 1
            layers.Flatten(),                                     
            layers.Dense(wandb.config.neurons, activation="relu"),             
            
            # Output 
            layers.Dense(n_classes, activation="softmax")     
        ]
    )

   model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
   
   model.fit(X_train, y_train, epochs=config.epochs, batch_size=config.bs,
             validation_split=0.1, callbacks=[WandbCallback()])
   

In [ ]:
wandb.agent(sweep_id, function=train)

In [ ]:
wandb.run.save
wandb.run.finish()

In [ ]:
# # Create model

# embed_len = 512
# max_tokens = 300
# max_words = 100
# n_classes = 2

# def create_model(f, neuron):
    
#     model = keras.Sequential(
#         [
#             keras.Input(shape=(max_tokens, )),
            
#             layers.Embedding(input_dim=max_words, 
#                              output_dim=embed_len,  
#                              input_length=max_tokens),
            
#             # Block 1
#             layers.Conv1D(f, kernel_size=(3, 3), activation="relu"),
#             layers.MaxPooling1D(pool_size=(3, 3)),
#             layers.Dropout(0.3),                  
            
#             # Block 2
#             layers.Conv1D(f, kernel_size=(3, 3), activation="relu"),
#             layers.MaxPooling1D(pool_size=(3, 3)),
#             layers.Dropout(0.3),                
            
#             # FC 1
#             layers.Flatten(),                                     
#             layers.Dense(neuron, activation="relu"),             
            
#             # Output 
#             layers.Dense(n_classes, activation="softmax")     
#         ]
#     )
    
#     model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

#     return model

In [ ]:
# model = KerasClassifier(build_fn=create_model,
#                         f=[128,256],
#                         epochs=[50],
#                         neuron=[32,64],
#                         batch_size=[32,64],)

# # Model params
# params={'batch_size':[32,64], 
#         'epochs':[50],
#         'neuron':[32,64],
#         'f':[128,256],
#         }

# # Grid search cv
# gs = GridSearchCV(estimator=model, 
#                   param_grid=params, 
#                   cv=10,
#                   refit=True)
# # Fit
# gs = gs.fit(X_train, y_train)

# print('Best acc:', gs.best_score_)
# print('Best params:', gs.best_params_)

# mean = gs.cv_results_['mean_test_score']
# stds = gs.cv_results_['std_test_score']
# params = gs.cv_results_['params']

# for mean, stdev, param in zip(mean, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))


##### Training best model

In [ ]:
# Retrain with best parameters

wandb.init(project="Title classifier", reinit=True)
wandb.run.name = 'Best params CNN model training'

model = keras.Sequential(
    [
        keras.Input(shape=(max_tokens, )),
        
        layers.Embedding(input_dim=max_words, 
                    output_dim=embed_len,  
                    input_length=max_tokens),
        
        # Block 1
        layers.Conv1D(256, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling1D(pool_size=(3, 3)),                  
        layers.Dropout(0.3),                  

        # Block 2
        layers.Conv1D(256, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling1D(pool_size=(3, 3)),                
        layers.Dropout(0.3),                  

        # FC 1
        layers.Flatten(),                                     
        layers.Dense(64, activation="relu"),                
        
        # Output 
        layers.Dense(n_classes, activation="softmax")  
    ]
)

# View summary
model.summary()

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Reduce lr
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=1e-6)
# Train
history = model.fit(X_train, y_train, epochs=50, batch_size=32,
                    validation_split=0.1, callbacks=[reduce_lr, WandbCallback()])

wandb.run.save
wandb.run.finish()

In [ ]:
# # View plots
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper left')
# plt.show()

# # Plot training & validation loss values
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper left')
# plt.show()

In [ ]:
# View loss

# y_test_p = np.argmax(model.predict(X_test), axis=-1)
y_test_p = model.predict(X_test)

test_loss, test_acc = model.evaluate(X_test, y_test_p, verbose=1)
print("Test loss:", test_loss)
print("Test accuracy:", test_acc)

In [ ]:
# # Prediction
# # y_test_p = np.argmax(model.predict(X_test), axis=-1)

# print(f"Classification report:\n"
#       f"{classification_report(y_test, y_test_p)}\n")

##### Export model for deployment

In [ ]:
model_dir = './best_model.h5'
model.save(model_dir)